본 단계에서는 보완하면 좋은 피쳐들을 다음과 같이 4가지 생성했습니다.

    1. price_per_cl = price / volume 용량보정단가
    2. alcohol_density = alchohol * volume 술 강도와 총 알코올 함량
    3. flavour_keywords : 핵심 풍미 단어
    4. 희귀 인덱스


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import gc

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
%matplotlib inline

!pip install koreanize-matplotlib
import koreanize_matplotlib

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/week_hands_on_project5/whiskey/whisky_processed2.csv')
df.columns = df.columns.str.lower()
print(df.shape)
df.head()

(1100, 33)


,name,main_info,whisky_type,volume(cl),alcohol(%),price(£),style_body_last,style_richness_last,style_smoke_last,style_sweetness_last,characters,food_paring,bottler,bottling_decade_clean,bottler_canon,is_independent,bottling_year,age,age_,age_int,is_nas,country,region,casktype,cask_group,colouring,vintage,vintage_clean,nose,palate,finish,reviews,_text_all
0,Blended Scotch TB-BSW6 Year Old Thompson Bros,blended scotch whisky,blended,70.0,46.0,34.25,3.0,3.0,0.0,3.0,"Figs, Sultana, Cinnamon, Nutmeg, Toffee, Coffee",NaN,NaN,NaN,thompson bros,1,NaN,NaN,NaN,17.584416,1,scotland,unknown_scotland,not specified,unknown,False,NaN,NaN,"Plum sauce, miso, golden sultanas, cinder toff...","Golden sultanas, raisin loaf, buttered Jamaica...","Soft warm spice; allspice, nutmeg and cinnamon...","Stunning stuff for the money, might well be my...",figs sultana cinnamon nutmeg toffee coffee plu...
1,Johnnie Walker Blue Label,blended scotch whisky,blended,70.0,40.0,158.00,3.0,3.0,1.0,2.0,"Rose, Hazelnut, Honey, Smoke, Dark Chocolate, ...","Octopus, Monkfish, Fondue",NaN,NaN,distillery bottling,0,NaN,NaN,NaN,17.584416,1,scotland,unknown_scotland,not specified,unknown,True,NaN,NaN,"Smooth, silky cereal, syrupy flapjacks, polish...",A silky caress. Unexpectedly strong spicy entr...,A perfectly-integrated slow fade of the palate...,Un elixir. || Excellent Whisky. Smooth on the ...,rose hazelnut honey smoke dark chocolate tobac...
2,The Big Ben,blended scotch whisky,blended,50.0,40.0,23.50,3.1,3.0,0.8,3.0,no description,NaN,NaN,NaN,distillery bottling,0,NaN,NaN,NaN,17.584416,1,scotland,unknown_scotland,not specified,unknown,True,NaN,NaN,"Medium weight, malty nose, with hints of orang...","Medium depth, hints of smokiness and sweetness...",Elegant finish.,Very pleased with the whiskey. Highly recommend,no description medium weight malty nose with h...
3,J&B Rare,blended scotch whisky,blended,70.0,40.0,25.25,2.8,2.7,0.2,2.8,no description,NaN,NaN,NaN,distillery bottling,0,NaN,NaN,NaN,17.584416,1,scotland,unknown_scotland,not specified,unknown,True,NaN,NaN,no description,no description,no description,Very nice || Was delighted to find this whisky...,no description no description no description n...
4,Dimple 15 Year OldLitre,blended scotch whisky,blended,100.0,43.0,69.95,3.0,3.0,0.0,3.0,"Caramel, Cinnamon, Vanilla, Dark Chocolate",NaN,NaN,NaN,distillery bottling,0,NaN,15 year old,15.0,15.000000,0,scotland,unknown_scotland,not specified,unknown,Unknown,NaN,NaN,no description,no description,no description,NaN,caramel cinnamon vanilla dark chocolate no des...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1100 entries, 0 to 1099
Data columns (total 33 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   name                   1100 non-null   object 
 1   main_info              1100 non-null   object 
 2   whisky_type            1100 non-null   object 
 3   volume(cl)             1100 non-null   float64
 4   alcohol(%)             1100 non-null   float64
 5   price(£)               1100 non-null   float64
 6   style_body_last        1100 non-null   float64
 7   style_richness_last    1100 non-null   float64
 8   style_smoke_last       1100 non-null   float64
 9   style_sweetness_last   1100 non-null   float64
 10  characters             1100 non-null   object 
 11  food_paring            30 non-null     object 
 12  bottler                505 non-null    object 
 13  bottling_decade_clean  178 non-null    float64
 14  bottler_canon          1100 non-null   object 
 15  is_i

In [ ]:
# price_per_cl = price / volume 용량보정단가

df['price_per_cl'] = df['price(£)'] / df['volume(cl)']

In [ ]:
# alcohol_density = alchohol * volume 술 강도와 총 알코올 함량

df['alcohol_density'] = df['alcohol(%)'] * df['volume(cl)']

In [ ]:
# flavour_keywords : 핵심 맛 단어

flavour_families = {
    "smoky_family": ["peaty", "smoky", "ash", "turf", "medicinal", "tar", "iodine", "charcoal", "soot", "bonfire"],
    "fruity_family": ["fruity", "apple", "pear", "peach", "apricot", "plum", "grape", "raisin", "sultana", "date",
                      "fig", "berry", "blackberry", "blueberry", "strawberry", "raspberry", "cherry", "citrus",
                      "orange", "lemon", "lime", "grapefruit", "tropical", "pineapple", "mango", "papaya", "melon",
                      "banana"],
    "floral_family": ["floral", "rose", "violet", "lavender", "jasmine", "heather", "honeysuckle"],
    "herbal_family": ["herbal", "mint", "thyme", "sage", "eucalyptus", "grass", "hay", "tea", "green"],
    "sweet_family": ["vanilla", "caramel", "toffee", "honey", "maple", "molasses", "syrup", "butterscotch",
                     "chocolate", "cocoa", "coffee"],
    "nutty_family": ["nutty", "almond", "hazelnut", "walnut", "pecan", "marzipan", "praline"],
    "spicy_family": ["spicy", "pepper", "black pepper", "white pepper", "clove", "cinnamon", "nutmeg",
                     "ginger", "cardamom", "anise", "licorice", "fennel", "chili"],
    "woody_family": ["oak", "wood", "cedar", "sandalwood", "pine", "resin"],
    "earthy_family": ["earthy", "moss", "damp", "leather", "tobacco", "cigar"],
    "malty_family": ["malty", "cereal", "grain", "barley", "biscuit", "cookie", "bread", "yeast"],
    "creamy_family": ["buttery", "creamy", "custard", "yogurt"],
    "wine_family": ["sherry", "port", "wine", "brandy", "rum", "cognac", "bourbon",
                    "oloroso", "pedro ximenez", "px"],
    "marine_family": ["salty", "briny", "seaweed", "coastal", "marine"],
    "other_family": ["oily", "rich", "dry", "smooth", "balanced", "complex", "elegant"]
}
import re

def detect_family(text, keywords):
  text = text.lower()
  return 1 if any(re.search(rf"\b{re.escape(kw)}\b", text) for kw in keywords) else 0

for fam, keywords in flavour_families.items():
    df[fam] = df["_text_all"].apply(lambda x: detect_family(x, keywords))


In [ ]:

def compute_rarity_scores_nostyle(
    df: pd.DataFrame,
    *,
    current_year: int = 2025,
    weights: dict | None = None,
    rare_regions: set | None = None,
    cask_strength_abv: float = 55.0,
    small_volume_cl: float = 70.0
) -> pd.DataFrame:
    df = df.copy()

    # helpers
    def c(name): return df[name] if name in df.columns else pd.Series(np.nan, index=df.index)
    def to_num(s): return pd.to_numeric(s, errors="coerce")
    def bool01(s): return to_num(s).fillna(0).clip(0, 1)
    def minmax(x):
        x = to_num(x)
        if x.notna().sum() <= 1:
            return pd.Series(0.0, index=x.index)
        lo, hi = np.nanpercentile(x, 1), np.nanpercentile(x, 99)
        x = x.clip(lo, hi)
        rng = (hi - lo) if hi > lo else 1.0
        out = (x - lo) / rng
        return out

    # pick columns
    price      = c("price(£)")
    abv        = c("alcohol(%)")
    volume     = c("volume(cl)")
    is_nas     = bool01(c("is_nas"))
    is_ind     = bool01(c("is_independent"))
    bottler    = c("bottler_canon").astype("string").str.strip().str.lower()
    region     = c("region").astype("string").str.strip().str.lower()
    country    = c("country").astype("string").str.strip().str.lower()
    bott_year  = to_num(c("bottling_year"))
    decade     = to_num(c("bottling_decade_clean"))
    age_txt    = to_num(c("age_"))
    vintage    = to_num(c("vintage_clean"))
    colouring  = c("colouring").astype("string").str.strip().str.lower()

    # 1) Scarcity (is_nas = 0, 독립병입 높을수록, 희귀지역 높을수록, 병입자 희소성 높을수록)
    bcounts = bottler.value_counts(dropna=True)
    inv_freq = bottler.map(lambda x: 1.0 / bcounts.get(x, np.nan))
    bottler_rarity = minmax(inv_freq).fillna(0)

    if rare_regions is None:
        rare_regions = {"campbeltown", "taiwan", "india"}
    rare_region_score = region.isin(rare_regions)
  # 결측치 처리
    rare_region_score = rare_region_score.fillna(False).astype(float)

    scarcity = ((1 - is_nas).fillna(0)
                + is_ind.fillna(0)
                + rare_region_score.fillna(0)
                + bottler_rarity.fillna(0))
    scarcity = minmax(scarcity).fillna(0)

    # 2) Price Premium (가격 높을수록, 캐스크 스트렝스 높을수록, 소용량 높을수록)
    price_norm = minmax(price).fillna(0)
    cask_strength_bonus = (to_num(abv) >= cask_strength_abv).fillna(False).astype(float)
    small_volume_bonus  = (to_num(volume) < small_volume_cl).fillna(False).astype(float)
    price_premium = (price_norm
                     + 0.5 * cask_strength_bonus
                     + 0.3 * small_volume_bonus)
    price_premium = minmax(price_premium).fillna(0)

    # 3) Distillery Factor (오래된 병입년도 높을수록, 80s 이전 높을수록, 무착색 높을수록)
    year_score = (current_year - bott_year).clip(lower=0)
    year_score = minmax(year_score).fillna(0)
    decade_bonus = (decade < 1990).fillna(False).astype(float)
    no_colour_bonus = (colouring == "no").fillna(False).astype(float)
    distillery_factor = (year_score
                         + 0.7 * decade_bonus
                         + 0.5 * no_colour_bonus)
    distillery_factor = minmax(distillery_factor).fillna(0)

    # 4) Vintage Value (오래된 빈티지 높을수록, 숙성연수 높을수록)
    age_norm = minmax(age_txt).fillna(0)
    vintage_recency = (current_year - vintage).clip(lower=0)
    vintage_old = minmax(vintage_recency).fillna(0)
    vintage_value = minmax(0.6 * vintage_old + 0.8 * age_norm).fillna(0)


# 최종 희귀도 계산 (4개 요소)
    w = {"scarcity": 0.30, "price_premium": 0.30, "distillery_factor": 0.20, "vintage_value": 0.20}
    if weights: w.update(weights)

    final = (w["scarcity"] * scarcity
             + w["price_premium"] * price_premium
             + w["distillery_factor"] * distillery_factor
             + w["vintage_value"] * vintage_value)
    final = minmax(final).fillna(0)

    df["rarity_scarcity"]      = scarcity
    df["rarity_price_premium"] = price_premium
    df["rarity_distillery"]    = distillery_factor
    df["rarity_vintage"]       = vintage_value
    df["rarity_score"]         = final
    return df


df = compute_rarity_scores_nostyle(df)
df[["name","rarity_scarcity","rarity_price_premium","rarity_distillery",
    "rarity_vintage","rarity_score"]].head()

,name,rarity_scarcity,rarity_price_premium,rarity_distillery,rarity_vintage,rarity_score
0,Blended Scotch TB-BSW6 Year Old Thompson Bros,0.399672,0.000750,0.0,0.000000,0.237533
1,Johnnie Walker Blue Label,0.000000,0.009766,0.0,0.000000,0.005793
2,The Big Ben,0.000000,0.273168,0.0,0.000000,0.162045
3,J&B Rare,0.000000,0.000095,0.0,0.000000,0.000056
4,Dimple 15 Year OldLitre,0.333333,0.003351,0.0,0.312207,0.323193


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1100 entries, 0 to 1099
Data columns (total 54 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   name                   1100 non-null   object 
 1   main_info              1100 non-null   object 
 2   whisky_type            1100 non-null   object 
 3   volume(cl)             1100 non-null   float64
 4   alcohol(%)             1100 non-null   float64
 5   price(£)               1100 non-null   float64
 6   style_body_last        1100 non-null   float64
 7   style_richness_last    1100 non-null   float64
 8   style_smoke_last       1100 non-null   float64
 9   style_sweetness_last   1100 non-null   float64
 10  characters             1100 non-null   object 
 11  food_paring            30 non-null     object 
 12  bottler                505 non-null    object 
 13  bottling_decade_clean  178 non-null    float64
 14  bottler_canon          1100 non-null   object 
 15  is_i

In [ ]:
df.to_csv('/content/drive/MyDrive/week_hands_on_project5/whiskey/whisky_engineered2.csv',index=False)